[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/jupyter/transformers/HuggingFace%20in%20Spark%20NLP%20-%20XLM-RoBERTa.ipynb)

## Import ALBERT models from HuggingFace 🤗  into Spark NLP 🚀 

Let's keep in mind a few things before we start 😊 

- This feature is only available in `Spark NLP 3.1.1` and above. So please make sure you have upgraded to the latest Spark NLP release
- You can import models for ALBERT from HuggingFace but they have to be compatible with `TensorFlow` and they have to be in `Fill Mask` category. Meaning, you cannot use ALBERT models trained/fine-tuned on a specific task such as token/sequence classification.
- Some of the `ALBERT` are larger than 2GB which is the limit in Spark NLP. Unfortunately, the models must be less than 2GB in size

## Export and Save HuggingFace model

- Let's install `HuggingFace` and `TensorFlow`. You don't need `TensorFlow` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock TensorFlow on `2.4.1` version and Transformers on `4.6.1`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.
- AlbertTokenizer requires the `SentencePiece` library, so we install that as well

In [1]:
!pip install -q transformers==4.6.1 tensorflow==2.4.1 sentencepiece

     |████████████████████████████████| 2.3MB 6.6MB/s 
     |████████████████████████████████| 394.3MB 37kB/s 
     |████████████████████████████████| 1.2MB 31.8MB/s 
     |████████████████████████████████| 3.3MB 37.2MB/s 
     |████████████████████████████████| 901kB 44.6MB/s 
     |████████████████████████████████| 471kB 60.5MB/s 
     |████████████████████████████████| 2.9MB 27.7MB/s 
     |████████████████████████████████| 3.8MB 16.2MB/s 


- HuggingFace comes with a native `saved_model` feature inside `save_pretrained` function for TensorFlow based models. We will use that to save it as TF `SavedModel`.
- We'll use [albert-base-v2](https://huggingface.co/albert-base-v2) model from HuggingFace as an example
- In addition to `TFAlbertModel` we also need to save the `AlbertTokenizer`. This is the same for every model, these are assets needed for tokenization inside Spark NLP.
- Since `albert-base-v2` model is PyTorch we will use `from_pt=True` param to convert it to TensorFlow

In [2]:
from transformers import AlbertTokenizer, TFAlbertModel
import tensorflow as tf

# albert-base-v2
MODEL_NAME = 'albert-base-v2'

AlbertTokenizer.from_pretrained(MODEL_NAME, return_tensors="pt").save_pretrained("./{}_tokenizer".format(MODEL_NAME))
TFAlbertModel.from_pretrained(MODEL_NAME, from_pt=True).save_pretrained("./{}".format(MODEL_NAME), saved_model=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFAlbertModel: ['predictions.decoder.bias', 'predictions.LayerNorm.bias', 'predictions.bias', 'predictions.dense.weight', 'predictions.dense.bias', 'predictions.LayerNorm.weight', 'predictions.decoder.weight']
- This IS expected if you are initializing TFAlbertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFAlbertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFAlbertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFAlbertModel for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

INFO:tensorflow:Assets written to: ./albert-base-v2/saved_model/1/assets


INFO:tensorflow:Assets written to: ./albert-base-v2/saved_model/1/assets


Let's have a look inside these two directories and see what we are dealing with:

In [3]:
!ls -l {MODEL_NAME}

total 45688
-rw-r--r-- 1 root root      791 Jun 23 14:54 config.json
drwxr-xr-x 3 root root     4096 Jun 23 14:54 saved_model
-rw-r--r-- 1 root root 46773496 Jun 23 14:54 tf_model.h5


In [4]:
!ls -l {MODEL_NAME}/saved_model/1

total 9416
drwxr-xr-x 2 root root    4096 Jun 23 14:54 assets
-rw-r--r-- 1 root root 9633593 Jun 23 14:54 saved_model.pb
drwxr-xr-x 2 root root    4096 Jun 23 14:54 variables


In [5]:
!ls -l {MODEL_NAME}_tokenizer

total 752
-rw-r--r-- 1 root root    244 Jun 23 14:53 special_tokens_map.json
-rw-r--r-- 1 root root 760289 Jun 23 14:53 spiece.model
-rw-r--r-- 1 root root    641 Jun 23 14:53 tokenizer_config.json


- as you can see, we need the SavedModel from `saved_model/1/` path
- we also be needing `spiece.model` file from the tokenizer
- all we need is to copy `spiece.model` file into `saved_model/1/assets` which Spark NLP will look for

In [6]:
# let's copy spiece.model file to saved_model/1/assets
!cp {MODEL_NAME}_tokenizer/spiece.model {MODEL_NAME}/saved_model/1/assets

## Import and Save ALBERT in Spark NLP


- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [7]:
! wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2021-06-23 14:54:18--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2021-06-23 14:54:19--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1608 (1.6K) [text/plain]
Saving to: ‘STDOUT’

-                   100%[===================>]   1.57K  --.-KB/s    in 0s      

2021-06-23 14:54:19 (34.3 MB/s) - written to stdout [1608/1608]

setup Cola

Let's start Spark with Spark NLP included via our simple `start()` function

In [8]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

- Let's use `loadSavedModel` functon in `AlbertEmbeddings` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `AlbertEmbeddings` in runtime, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- `setStorageRef` is very important. When you are training a task like NER or any Text Classification, we use this reference to bound the trained model to this specific embeddings so you won't load a different embeddings by mistake and see terrible results 😊
- It's up to you what you put in `setStorageRef` but it cannot be changed later on. We usually use the name of the model to be clear, but you can get creative if you want! 
- The `dimension` param is is purely cosmetic and won't change anything. It's mostly for you to know later via `.getDimension` what is the dimension of your model. So set this accordingly.


In [9]:
from sparknlp.annotator import *

albert = AlbertEmbeddings.loadSavedModel(
     '{}/saved_model/1'.format(MODEL_NAME),
     spark
 )\
 .setInputCols(["sentence",'token'])\
 .setOutputCol("embeddings")\
 .setCaseSensitive(False)\
 .setDimension(768)\
 .setStorageRef('albert_base_uncased') 

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [10]:
albert.write().overwrite().save("./{}_spark_nlp".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [11]:
!rm -rf {MODEL_NAME}_tokenizer {MODEL_NAME}

Awesome 😎  !

This is your ALBERT model from HuggingFace 🤗 loaded and saved by Spark NLP 🚀 

In [12]:
! ls -l {MODEL_NAME}_spark_nlp

total 43748
-rw-r--r-- 1 root root   760289 Jun 23 14:57 albert_spp
-rw-r--r-- 1 root root 44026723 Jun 23 14:57 albert_tensorflow
drwxr-xr-x 3 root root     4096 Jun 23 14:56 fields
drwxr-xr-x 2 root root     4096 Jun 23 14:56 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny RoBERTa model 😊 

In [13]:
albert_loaded = AlbertEmbeddings.load("./{}_spark_nlp".format(MODEL_NAME))\
  .setInputCols(["sentence",'token'])\
  .setOutputCol("embeddings")\
  .setCaseSensitive(False)

In [14]:
albert_loaded.getStorageRef()

'albert_base_uncased'

That's it! You can now go wild and use hundreds of ALBERT models from HuggingFace 🤗 in Spark NLP 🚀 
